# Import modules

In [ ]:
import stlab
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from scipy.optimize import *
import time
import sys 
import io
import os
from IPython.display import display, Javascript
from shutil import *
from stlab.devices.RS_SGS100A import RS_SGS100A
from stlab.devices.Keysight_N5183B import Keysight_N5183B

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration_BMDevice import config, RR_1_IF,RO_lo, readout_len, Q1_lo, Q1_IF, sat_Q1_len, gauss_len, R1_RS, Q1_RS

In [ ]:
from scipy.signal import gaussian
print(readout_len)

#Analog output must be between -0.5 and 0.5-2^-16. Therefore, set amp here to at most 0.25 if you want to scale by 2!
gauss_test = 0.24 * gaussian(gauss_len, 0.15* gauss_len)

gauss_pulse_len = 20  # nsec
Amp = 0.24  # Pulse Amplitude
gauss_arg = np.linspace(-3, 3, gauss_pulse_len)
gauss_wf = np.exp(-(gauss_arg ** 2) / 2)
gauss_wf = Amp * gauss_wf / np.max(gauss_wf)

plt.plot(gauss_wf)
plt.plot(gauss_test)

# Edit configuration parameters

**To do**

# Define microwave sources

In [ ]:
RR = RS_SGS100A("TCPIP::169.254.184.193::INSTR", reset=True,verb=True) 
RR.EXTref()
RR.RFon()
RR.setCWpower(-15)
RR.setCWfrequency(RO_lo)
RR.write(':SOURce:IQ:IMPairment:LEAKage:I ' + R1_RS[0])
RR.write('SOURce:IQ:IMPairment:LEAKage:Q ' + R1_RS[1])
RR.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + R1_RS[2])
RR.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + R1_RS[3])
RR.IQon()
RR.write(':SOURce:IQ:IMPairment:STATe ON')

#LO for downconversion
MXG = Keysight_N5183B(addr='TCPIP::192.168.1.91::INSTR',reset=True,verb=True)
MXG.RFon()
MXG.setCWpower(19)
MXG.setCWfrequency(RO_lo)
MXG.INTref()

QDrive = RS_SGS100A("TCPIP::169.254.50.124::INSTR", reset=True,verb=True) 
QDrive.EXTref()
QDrive.RFon()
QDrive.setCWpower(0) 
QDrive.setCWfrequency(Q1_lo)
QDrive.write(':SOURce:IQ:IMPairment:LEAKage:I ' + Q1_RS[0])
QDrive.write('SOURce:IQ:IMPairment:LEAKage:Q ' + Q1_RS[1])
QDrive.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + Q1_RS[2])
QDrive.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + Q1_RS[3])
QDrive.IQon()
QDrive.write(':SOURce:IQ:IMPairment:STATe ON')

# Configure QM unit

In [ ]:
qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

# QUA Program

In [ ]:
def Time_Power_Rabi(n_max, f_if, a_vec, b, t_vec, QLO_Power, RR_1_IF):
    QDrive.setCWpower(QLO_Power)
    
    a_min = np.min(a_vec)
    a_max = np.max(a_vec)
    
    if len(a_vec) == 1: 
        da = 1 
    else:
        da = a_vec[1]-a_vec[0]
        
    t_min = int(np.min(t_vec))
    t_max = int(np.max(t_vec))
    dt = int(t_vec[1] - t_vec[0])
        
    with program() as time_power_Rabi:  
        ##############################
        # declare real-time variables:
        ##############################

        n = declare(int)        # Averaging
        f = declare(int)        # Frequencies

        f_arr = declare(fixed,value = f_if)
        f_i = declare(fixed)
        RR_IF = declare(int)
        assign(RR_IF,RR_1_IF)

        a=declare(fixed)
        t=declare(int)

        A=declare(fixed)
        B=declare(fixed)
        iA=declare(fixed)
        iB=declare(fixed)
        Re=declare(fixed)
        Im=declare(fixed)

        Re_st = declare_stream()
        Im_st = declare_stream()
        f_st = declare_stream()

        ###############
        # the sequence:
        ###############
        with for_(n, 0, n < n_max, n + 1):

            with for_each_(f_i, f_arr): 
                assign(f,Cast.mul_int_by_fixed(RR_IF, f_i))
                save(f,f_st)
                update_frequency("RR_1", f)

                with for_(a, a_min, a < a_max + da/2, a + da):

                    with for_(t, t_min, t <= t_max, t + dt):

                        wait(int(2500/4),"RR_1","Q1")
                        play("gaussian"*amp(a), "Q1", duration = t)
                        align("Q1", "RR_1")

                        measure("readout"*amp(b), "RR_1", None, demod.full("integW_cos", A, "out1"),
                                                                    demod.full("integW_sin",B,"out2"),
                                                                    demod.full("integW_sin", iA, "out1"),
                                                                    demod.full("integW_cos",iB,"out2"))

                        assign(Re, A + B)       
                        assign(Im, iA - iB)
                        save(Re, Re_st)
                        save(Im, Im_st)              

        with stream_processing():
            Re_st.buffer(len(f_vec),len(a_vec), len(t_vec)).average().save("Re")
            Im_st.buffer(len(f_vec),len(a_vec), len(t_vec)).average().save("Im")
            f_st.buffer(len(f_vec)).average().save("f")

    job = qm.execute(time_power_Rabi, duration_limit=0, data_limit=0)

    res_handles= job.result_handles
    res_handles.wait_for_all_values()

    Re_handle = res_handles.get("Re")
    Im_handle = res_handles.get("Im")
    f_handle = res_handles.get("f")

    Re_s = Re_handle.fetch_all()
    Im_s = Im_handle.fetch_all()
    f = f_handle.fetch_all()
    
    return Re_s, Im_s, f

# Define Parameters

In [ ]:
n_max = 10000

#Readout pulse amplitude
b = 0.24

#Qubit pulse time (in clock cycles of 4ns)
t_min = 6 
t_max = 125 
dt = 1
t_vec = np.arange(t_min,t_max+dt/2,dt)

# Qubit pulse amp 
a_min = 0
a_max = 2
da = 0.02
a_vec=np.arange(a_min,a_max+da/2,da)

f_e = 4.28445e9
f_g = 4.28995e9  #4.28935e9
f_d = 4.29935e9
f_vec = np.array([f_e,f_g,f_d])
f_if = ((f_vec-RO_lo)/(RR_1_IF)).tolist()
print(f_vec)
print(f_if)

#print(len(t_vec))
print(len(a_vec))
print(len(t_vec))
#print(a_vec)

In [ ]:
Re, Im, f = Time_Power_Rabi(n_max, f_if, a_vec, b, t_vec, 0, RR_1_IF)

## Fetch data and save to .dat

In [ ]:
prefix = 'S' #prefix for measurement folder name.  Can be anything or empty
idstring = f'Q1_Time_Power_Rabi'

f_arr = f[:,]+RO_lo
f_arrf = np.repeat(f_arr[:,:,np.newaxis],np.shape(Re)[-1],axis=2)
a_arr = ((a_vec.reshape(len(a_vec),1)+t_vec)-t_vec).T
a_arrf = np.repeat(a_arr[np.newaxis,:,:],np.shape(f_vec)[0],axis=0)
t_arr = ((t_vec.reshape(len(t_vec),1))+a_vec)-a_vec
t_arrf = np.repeat(t_arr[np.newaxis,:,:],np.shape(f_vec)[0],axis=0)

data_Re = Re
data_Im = Im
data_Sig = np.abs(data_Re + 1j*data_Im)
data_Amp = 20*(np.log10(np.abs(data_Sig)))
data_Ph = np.unwrap(np.arctan(data_Im/data_Re))

#data = np.asarray([((a_vec.reshape(len(a_vec),1)+t_vec)-t_vec).T,((t_vec.reshape(len(t_vec),1))+a_vec)-a_vec,data_Re,data_Im,data_Sig,data_Amp,data_Ph])
data = np.asarray([f_arrf,a_arrf,t_arrf,data_Re,data_Im,data_Sig,data_Amp,data_Ph])

for j,fm in enumerate(f_vec):
    print(j,fm)
    for i,tm in enumerate(t_vec):
        print(i,tm)
        data_dict={'Resonator Frequency (Hz)': data[0][j][i],
               'Qubit Pulse Amplitude':data[1][j][i],
               'Qubit Pulse Time (ns)':data[2][j][i]*4,
               'Real':data[3][j][i],
               'Imaginary':data[4][j][i],
               'Signal':data[5][j][i],
               'Amplitude (dB)':data[6][j][i],
               'Phase (rad)':data[7][j][i]
        }
        if (i==0 and j==0):
            old_stdout = sys.stdout
            new_stdout = io.StringIO()
            sys.stdout = new_stdout

            myfile=stlab.newfile(prefix,idstring,data_dict.keys(),autoindex=True, git_id= False)

            output = new_stdout.getvalue()
            sys.stdout = old_stdout
            print(output)
            M_ind = output.find("Measurement Name")
            M_name = output[M_ind+len('Measurement Name:  '):-1]
        stlab.savedict(myfile,data_dict)   #data_dict['Qubit Pulse Amplitude']
        stlab.metagen.fromarrays(myfile,f_vec[0:j+1],a_vec[0:i+1],t_vec[0:i+1]*4,xtitle='Resonator Frequency (Hz)',ytitle='Qubit Pulse Amplitude',ztitle='Qubit Pulse Time (ns)',colnames=list(data_dict))

# Processing/Plotting

In [ ]:
l = 0
sig = Re[l,:,:] + 1j*Im[l,:,:]

power = 20*(np.log10(np.abs(sig)))#+10*np.log10(1000/50)#np.abs(sig)   #equation from wiki
power2=power/np.mean(power,axis=0)                             #norm lbl
power3=(scipy.ndimage.gaussian_filter(1/(power/np.mean(power,axis=0)),[1,5])) #gaussian filtered norm lbl
power4=power-np.mean(power,axis=0)                                    #sub lbl
power5=scipy.ndimage.gaussian_filter(power,[1,3])-np.mean(power,axis=0)#gaussian filtered sub lbl
power6=scipy.ndimage.gaussian_filter(power,[1,3]) #just gaussian filtered
phase = np.unwrap(np.arctan(Im/Re))

plt.figure(num=None, figsize=(8, 6), dpi=100)
plt.tight_layout()
#plt.plot(RO_lo+f_vec,power)
#wait_time = 0 

X, Y = np.meshgrid(t_vec*4, a_vec)
#plt.contourf(X,Y, power.T,levels=100,norm=colors.PowerNorm(gamma=2.5),cmap='RdBu')
plt.pcolormesh(X,Y, power6,shading = 'auto',norm=colors.PowerNorm(gamma=2.5),cmap='RdBu')

plt.xlabel('Qubit Pulse Time (ns)')
plt.title('Time Power Rabi Sweep, t_RO={}ns, R_amp = {}'.format(readout_len, b))
# #plt.xticks(X[0,0],X[0,-1])
plt.ylabel('Qubit Pulse Amplitude')
cbar = plt.colorbar()
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Normalized Signal Amplitude', rotation=270);
# cbar = plt.colorbar(im, fraction=0.25, pad=0.04, );

In [ ]:
from scipy.signal import savgol_filter
from scipy.signal import find_peaks

amax_t = np.argmax(power6, axis = 0)
amin_t = np.argmin(power6, axis = 0)

power_t_max = np.zeros(len(t_vec))
power_t_min = np.zeros(len(t_vec))
for i in range(len(t_vec)): 
    power_t_max[i] = power6[amax_t[i],i]
    power_t_min[i] = power6[amin_t[i],i]
    
cont_t = power_t_max-power_t_min
cont_t_s = savgol_filter(cont_t, 13, 9)
plt.figure(figsize=(8,6),dpi=100)
plt.plot(t_vec*4,cont_t)
plt.plot(t_vec*4,cont_t_s)
plt.xlabel('Qubit Pulse Time (ns)')
plt.ylabel('Peak to peak amplitude (dB)')          #The difference between the maximum and minimum value of  
plt.xlim(t_vec[0]*4, t_vec[-1]*4)                  #the smoothed power at a specific qubit pulse time

max_c = find_peaks(cont_t_s)[0][0]
print(find_peaks(cont_t_s))

App = np.round(cont_t_s[max_c],3)
t_e = t_vec[max_c]*4
a_e = np.round(a_vec[np.argmin(power[:,max_c])],3)
a_ind_e = np.argmin(power[:,max_c])

print(f'Maximum peak to peak amplitude of {App} dB for t = {t_e} ns')
print(f'Corresponding amplitude is a = {a_e}')

In [ ]:
i = 80
j = np.argmin(power[:,max_c])

fig = plt.figure(figsize=(10,4),dpi=100,constrained_layout=True)
gs = GridSpec(1, 2, figure=fig)
ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1])

ax1.plot(t_vec*4,power.T[:,i],label =f'a={a_vec[i]}')
ax1.plot(t_vec*4,power.T[:,j],label =f'a={a_vec[j]}')
ax1.set_xlabel('Qubit Pulse Time (ns)')
ax1.set_ylabel('Normalized Signal Amplitude')
ax1.legend(loc = 'best');
#ax1.set_ylim(-86.5,-82.5);
ax1.set_xlim(t_vec[0]*4,t_vec[-1]*4);

k = max_c-5
l = max_c
ax2.plot(a_vec,power.T[k,:],label =f't={t_vec[k]*4} ns')
ax2.plot(a_vec,power.T[l,:],label =f't={t_vec[l]*4} ns')
ax2.set_xlabel('Qubit Pulse Amplitude')
ax2.set_ylabel('Normalized Signal Amplitude')
ax2.legend(loc = 'best');
#ax2.set_ylim(-86.5,-82.5);
ax2.set_xlim(a_vec[0],a_vec[-1]);

# Contrast

Idea: probe the resonator response at both $f_g$ and $f_e$, the resonator frequency when the qubit is in the ground or excited state respectively. Perform the Rabi sweeps and determine at which qubit pulse length and amplitude the response $A_e$ at $f_e$ is the largest. Find the corresponding response $A_g$ at $f_g$. Shift these amplitudes (in dB) to the 0 dB level in the following way:

$$
A_{g,n} = A_g - \max{(A_g,A_e,A_{\Delta})} \\
A_{e,n} = A_e - \max{(A_g,A_e,A_{\Delta})}.\\
$$
Here $A_{\Delta}$ is the resonator response at a frequency $f_{\Delta}$ far detuned from both $f_g$ and $f_e$. With these shifted amplitudes we calculate the contrast:

$$
C = \frac{A_{g,n}-A_{e,n}}{|A_{g,n}+A_{e,n}|},
$$

The contrast specificies the relative height between the resonator responses at its two dispersively shifted frequencies $f_g$ and $f_e$ and is a measure for the populations in the ground and excited states (here I assume a perfect two-level system, leakage to higher states is not captured by the contrast as a result of the chosen normalization). If all population is in the excited (ground) state $C = 1 (-1)$. If the population in the ground and excited state is equal $C = 0$.

**Note:** When changing the readout length, the linewidth of resonator dips increases a lot and $f_e$ tends to shift around a bit. When optimizing for readout parameters (pulse length, pulse amplitude, LO power) it is advised to first do a quick frequency sweep to determine $f_e$ (could maybe be automated in the future). 

In [ ]:
#Run power sweeps for 3 frequencies [f_e,f_g,f_Delta]
#Find max response at f_e
#Find corresponding response at f_g and f_Delta
#Give them to C function to calculate contrast

Ae = power6[a_ind_e,max_c]

sig_g = Re[1,:,:] + 1j*Im[1,:,:]
power_g = 20*(np.log10(np.abs(sig_g))) 
power6_g = scipy.ndimage.gaussian_filter(power_g,[1,3])
Ag = power6_g[a_ind_e,max_c]

sig_delta = Re[2,:,:] + 1j*Im[2,:,:]
power_delta = 20*(np.log10(np.abs(sig_delta))) 
power6_delta = scipy.ndimage.gaussian_filter(power_delta,[1,3])
Ad = power6_delta[a_ind_e,max_c]

Ae_n = Ae - np.max([Ae,Ag,Ad])
Ag_n = Ag - np.max([Ae,Ag,Ad])
#Ad_n = Ad - np.max([Ae,Ag,Ad])

def C(Ae,Ag):
    return (Ag-Ae)/np.abs(Ag+Ae)

print(f'The contrast is {np.round(C(Ae_n,Ag_n),4)}.')

# Save this file and configuration file to measurement folder

In [ ]:
#save notebook
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
#define document paths
meas_path = os.path.join(os.getcwd(),M_name)

current_nb_path = os.path.join(os.getcwd(),nb_name)
save_nb_path = os.path.join(meas_path,nb_name)

current_config_path = os.path.join(os.getcwd(), 'Configuration_BMDevice.py')
save_config_path = os.path.join(meas_path, 'Configuration_BMDevice.py')

#copy to measurement folder 
copy2(current_nb_path,save_nb_path);
copy2(current_config_path,save_config_path);

# Simulation

In [ ]:
a1 = 0.24
b1 = 1

with program() as sim_prog:
    a=declare(fixed)
    
    assign(a, a1)
    
    play("gaussian"*amp(b1), "Q1", duration = 50)
    #align("Q1", "RR_1")
    #wait(int((sat_Q1_len-readout_len/2)/4),"RR_1")
    #play("readout"*amp(a),"RR_1")
    #measure("readout"*amp(b), "RR_1", "raw_adc")
        

# In the OPX, the analog signal starts 184 after the play command. In order to simulate it, we added the same latency
# here, and this is the time_of_flight in the configuration file
job = qmm.simulate(config,
    sim_prog,
    SimulationConfig(
        500,
        include_analog_waveforms=True,
    ),latency=184
)

# get DAC and digital samples
samples = job.get_simulated_samples()

# plot all ports:
plt.figure(figsize = (8,6),dpi=100)
plt.plot(samples.con1.analog["1"])
plt.plot(samples.con1.analog["3"],alpha = 1)
plt.legend("analog 1")
plt.xlabel("Time [ns]")
plt.ylabel("Signal [V]")
plt.xlim(0)

# samples = job.get_simulated_samples()
# res = job.result_handles
# raw_adc = res.raw_adc_input1.fetch_all()["value"]

# ax1 = plt.subplot(211)
# plt.plot(samples.con1.analog["1"])
# plt.title("Simulated samples")
# plt.subplot(212, sharex=ax1)
# plt.plot(raw_adc / 2 ** 12)  # Converting the 12 bit ADC value to voltage
# plt.title("Raw ADC input")
# plt.xlabel("Sample number")
# plt.tight_layout()
# plt.show()